In [14]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
import altair as alt
import pandas as pd

In [ ]:
#https://vega.github.io/vega-lite/tutorials/explore.html
# USE THIS COLOUR PATETTE (end of page)
# https://altair-viz.github.io/gallery/selection_histogram.html
# https://altair-viz.github.io/user_guide/interactions.html
# https://altair-viz.github.io/gallery/iowa_electricity.html

In [15]:
ds = pd.read_csv('S1SubActivities_preprocessed.csv', index_col = None) 
ds.start = pd.to_datetime(ds.start, format='%Y-%m-%d %H:%M:%S')
ds.end = pd.to_datetime(ds.end, format='%Y-%m-%d %H:%M:%S')
ds = ds.sort_values('start')
ds.reset_index(drop = True, inplace = True)

In [16]:
ds.head(n=2)

,subActNum,subAct,start,end
0,67,bathroom_cabinet,2003-03-27 06:43:40,2003-03-27 06:43:43
1,100,bathroom_toiletflush,2003-03-27 06:44:06,2003-03-27 07:12:41


In [17]:
%run add_DAY_WDWE_phaseI.py

In [18]:
ds = add_DAY_WDWE_phaseI(ds)

In [19]:
ds.head(n=2)

,subActNum,subAct,start,end,dayNumeric,DAY,WDWE,HOUR,durationSec
0,67,bathroom_cabinet,2003-03-27 06:43:40,2003-03-27 06:43:43,3,Thu,WD,6,3.0
1,100,bathroom_toiletflush,2003-03-27 06:44:06,2003-03-27 07:12:41,3,Thu,WD,6,1715.0


In [11]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2772 entries, 0 to 2771
Data columns (total 8 columns):
subActNum      2772 non-null int64
subAct         2772 non-null object
start          2772 non-null datetime64[ns]
end            2772 non-null datetime64[ns]
DAY            2772 non-null object
WDWE           2772 non-null object
HOUR           2772 non-null int64
durationSec    2772 non-null float64
dtypes: datetime64[ns](2), float64(1), int64(2), object(3)
memory usage: 173.4+ KB


1. Set specified column as index
2. Extract dayofweek from index (ds.index.dayofweek.astype(str)), as this as an attribute
3. Replace '0' with 'mon' and so on
4. Duplic

In [20]:
# Unique subActNum list
unique_list = []
for x in ds.subAct:                     # Traverse for all elements 
    if x not in unique_list:            # Check if exists in unique_list or not 
        unique_list.append(x)     

In [ ]:
for subAct in unique_list:
    
    fig = px.histogram(ds[ds['subAct']==subAct], x="HOUR", y="HOUR", color="WDWE",
                   marginal="box", # or violin, rug
                   hover_data=dsCheck.columns)
    fig.show()

In [21]:
import plotly.express as px
tips = px.data.tips()
fig = px.histogram(dsCheck, x="HOUR", y="HOUR", color="WDWE",
                   marginal="box", # or violin, rug
                   hover_data=dsCheck.columns)
fig.show()

NameError: name 'dsCheck' is not defined

In [ ]:
dsCheck = ds[ds['subAct']=='bathroom_cabinet']

In [ ]:
dsCheck.head(n=5)

In [ ]:
len(dsCheck)

In [ ]:
import plotly.graph_objects as go

import numpy as np

x = dsCheck['deltaStartEnd']

fig = go.Figure(data=[go.Histogram(x=x)])
fig.show()

In [ ]:
import plotly.graph_objects as go

import numpy as np

x = dsCheck['HOUR']

fig = go.Figure(data=[go.Histogram(x=x)])
fig.show()

In [ ]:
import plotly.graph_objects as go

import numpy as np

x = dsCheck['HOUR']

fig = go.Figure(data=[go.Histogram(x=x, histnorm='probability')])
fig.show()

In [ ]:
alt.Chart(source).transform_timeunit(Month='month(date)').transform_joinaggregate(
    mean_temp='mean(temp_max)', 
    groupby=['Month'].transform_bin(['bin_max', 'bin_min'], 'temp_max').transform_aggregate(
    value='count()', groupby=['Month', 'mean_temp', 'bin_min', 'bin_max']).transform_impute(
    impute='value', groupby=['Month', 'mean_temp'], key='bin_min', value=0).mark_area(
    interpolate='monotone', fillOpacity=0.8, stroke='lightgray', strokeWidth=0.5).encode(
    alt.X('bin_min:Q', 
          bin='binned', 
          title='Maximum Daily Temperature (C)'),
    alt.Y('value:Q', 
          scale=alt.Scale(range = [step, -step * overlap]), axis=None), 
    alt.Fill('mean_temp:Q',
        legend=None,
        scale=alt.Scale(domain=[30, 5], scheme='redyellowblue')),
    alt.Row('Month:T',
        title=None,
        header=alt.Header(labelAngle=0, labelAlign='right', format='%B'))).properties(
    bounds='flush', title='Seattle Weather', height=step).configure_facet(
    spacing=0).configure_view(
    stroke=None).configure_title(
    anchor='end')

In [ ]:
step = 40
overlap = 0

alt.Chart(ds
).transform_bin(
    ['bin_max', 'bin_min'], 'deltaStartEnd'
).transform_aggregate(
    value='count()', groupby=['HOUR', 'deltaStartEnd', 'bin_min', 'bin_max']
).transform_impute(
    impute='value', groupby=['HOUR', 'deltaStartEnd'], key='bin_min', value=0
).mark_area(
    interpolate='monotone',
    fillOpacity=0.8,
    stroke='black',
    strokeWidth=0.5
).encode(
    alt.X('bin_min:Q', bin='binned', title='Delta'),
    alt.Y(
        'value:Q',
        scale=alt.Scale(range=[step, -step * overlap]),
        axis=None
    ),
    alt.Row(
        'HOUR:N',
        title=None,
        header=alt.Header(labelAngle=0, labelAlign='right', format='%B')
    )
).properties(
    bounds='flush', title='67-100', height=step
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
).configure_title(
    anchor='end'
)


In [ ]:
step = 40
overlap = 0

alt.Chart(ds
).transform_joinaggregate(
    mean_delta='mean(deltaStartEnd)', groupby=['subAct']
).transform_bin(
    ['bin_max', 'bin_min'], 'deltaStartEnd'
).transform_aggregate(
    value='count()', groupby=['subAct', 'deltaStartEnd', 'bin_min', 'bin_max']
).transform_impute(
    impute='value', groupby=['subAct', 'deltaStartEnd'], key='bin_min', value=0
).mark_area(
    interpolate='monotone', fillOpacity=0.8, stroke='black', strokeWidth=0.5
).encode(
    alt.X('bin_min:Q', bin='binned', title='Delta'),
    alt.Y(
        'value:N',
        scale=alt.Scale(range=[step, -step * overlap]),
        axis=None),
    alt.Fill('mean_delta:Q',
        legend=None,
        scale=alt.Scale(domain=[30000, 5], scheme='redyellowblue')),
    alt.Row(
        'HOUR:N',
        title=None,
        header=alt.Header(labelAngle=0, labelAlign='right', format='%B')
    )
).properties(
    bounds='flush', title='67-100', height=step
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
).configure_title(
    anchor='end'
).interactive()

In [ ]:
step = 40
overlap = 0

alt.Chart(ds
).transform_joinaggregate(
    mean_delta='mean(deltaStartEnd)', groupby=['subAct']
).transform_bin(
    ['bin_max', 'bin_min'], 'deltaStartEnd'
).transform_aggregate(
    value='count()', groupby=['HOUR', 'deltaStartEnd', 'bin_min', 'bin_max']
).transform_impute(
    impute='value', groupby=['subAct', 'deltaStartEnd'], key='bin_min', value=0
).mark_area(
    interpolate='monotone', fillOpacity=0.8, stroke='black', strokeWidth=0.5
).encode(
    alt.X('bin_min:Q', bin='binned', title='Delta'),
    alt.Y(
        'value:N',
        scale=alt.Scale(range=[step, -step * overlap]),
        axis=None),
    alt.Fill('mean_delta:Q',
        legend=None,
        scale=alt.Scale(domain=[30000, 5], scheme='redyellowblue')),
    alt.Row(
        'HOUR:N',
        title=None,
        header=alt.Header(labelAngle=0, labelAlign='right', format='%B')
    )
).properties(
    bounds='flush', title='67-100', height=step
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
).configure_title(
    anchor='end'
).interactive()


In [ ]:
step = 40
overlap = 0

alt.Chart(ds
).transform_bin(
    ['bin_max', 'bin_min'], 'deltaStartEnd'
).transform_aggregate(
    value='count()', groupby=['HOUR', 'deltaStartEnd', 'bin_min', 'bin_max']
).transform_impute(
    impute='value', groupby=['subAct', 'deltaStartEnd'], key='bin_min', value=0
).mark_area(
    interpolate='monotone',
    fillOpacity=0.8,
    stroke='black',
    strokeWidth=0.5
).encode(
    alt.X('bin_min:Q', bin='binned', title='Delta'),
    alt.Y(
        'value:N',
        scale=alt.Scale(range=[step, -step * overlap]),
        axis=None
    ),
    alt.Row(
        'HOUR:N',
        title=None,
        header=alt.Header(labelAngle=0, labelAlign='right', format='%B')
    )
).properties(
    bounds='flush', title='67-100', height=step
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
).configure_title(
    anchor='end'
)


* Title with SubActNum and SubAct
* Start Date as Thu 03 March
* Weekends orange
* Weekdays blue
* 24till2
* 3till5
* 6till8
* 9till11
* 12till14
* 15till17
* 18till20
* 21till23

In [ ]:
# Unique subActNum list
unique_list = []
for x in ds.subActNum:                     # Traverse for all elements 
    if x not in unique_list:            # Check if exists in unique_list or not 
        unique_list.append(x)     

In [ ]:

step = 20
overlap = 1

alt.Chart(ds).transform_joinaggregate(
    mean_duration = 'mean(deltaStartEnd)', 
    groupby=['subActNum']).transform_bin(['bin_max', 'bin_min'], 'max(deltaStartEnd)').transform_aggregate(
    value='count()', groupby=['subActNum', 'mean_duration', 'bin_min', 'bin_max']).transform_impute(
    impute='value', groupby=['subActNum', 'mean_duration'], key='bin_min', value=0).mark_area(
    interpolate='monotone', fillOpacity=0.8, stroke='lightgray', strokeWidth=0.5).encode(
    alt.X('bin_min:Q', 
          bin='binned', 
          title='Maximum Daily Temperature (C)'),
    alt.Y('value:Q', 
          scale=alt.Scale(range = [step, -step * overlap]), axis=None), 
    alt.Fill('mean_duration:Q',
        legend=None,
        scale=alt.Scale(domain=[30, 5], scheme='redyellowblue')),
    alt.Row('date(start):T',
        title=None,
        header=alt.Header(labelAngle=0, labelAlign='right', format='%B'))).properties(
    bounds='flush', title='Seattle Weather', height=step).configure_facet(
    spacing=0).configure_view(
    stroke=None).configure_title(
    anchor='end')


In [ ]:
# https://vega.github.io/vega-lite/data/seattle-weather.csv
# 
alt.Chart(source).transform_timeunit(Month='month(date)').transform_joinaggregate(
    mean_temp='mean(temp_max)', 
    groupby=['Month'].transform_bin(['bin_max', 'bin_min'], 'temp_max').transform_aggregate(
    value='count()', groupby=['Month', 'mean_temp', 'bin_min', 'bin_max']).transform_impute(
    impute='value', groupby=['Month', 'mean_temp'], key='bin_min', value=0).mark_area(
    interpolate='monotone', fillOpacity=0.8, stroke='lightgray', strokeWidth=0.5).encode(
    alt.X('bin_min:Q', 
          bin='binned', 
          title='Maximum Daily Temperature (C)'),
    alt.Y('value:Q', 
          scale=alt.Scale(range = [step, -step * overlap]), axis=None), 
    alt.Fill('mean_temp:Q',
        legend=None,
        scale=alt.Scale(domain=[30, 5], scheme='redyellowblue')),
    alt.Row('Month:T',
        title=None,
        header=alt.Header(labelAngle=0, labelAlign='right', format='%B'))).properties(
    bounds='flush', title='Seattle Weather', height=step).configure_facet(
    spacing=0).configure_view(
    stroke=None).configure_title(
    anchor='end')

In [ ]:
ds.head(n=5)

In [ ]:
# https://altair-viz.github.io/gallery/binned_heatmap.html
alt.Chart(ds).mark_rect().encode(
    alt.X('DAY'),
    alt.Y('subAct'),
    alt.Color('deltaStartEnd:Q', scale=alt.Scale(scheme='greenblue'))
).properties(width=550).interactive()

In [ ]:
# DS Norm
ds['durationNorm'] = (ds['deltaStartEnd'] - min(ds['deltaStartEnd'])
                     )/(max(ds['deltaStartEnd']) - min(ds['deltaStartEnd']))

In [ ]:
ds.head(n=5)

In [ ]:
# https://altair-viz.github.io/gallery/binned_heatmap.html
alt.Chart(ds).mark_rect().encode(
    alt.X('DAY'),
    alt.Y('subAct'),
    alt.Color('durationNorm:Q', scale=alt.Scale(scheme='inferno'))
).properties(width=550).interactive()

In [ ]:
alt.Chart(ds).mark_rect().encode(
    alt.X('date(start):O'),
    alt.Y('subAct'),
    alt.Color('durationNorm:Q', scale=alt.Scale(scheme='inferno'))
).properties(width=550).interactive()

In [ ]:
alt.Chart(ds).mark_rect().encode(
    alt.X('date(start):O'),
    alt.Y('subAct'),
    alt.Color('mean(deltaStartEnd)', scale=alt.Scale(scheme='inferno'))
).properties(width=550).interactive()

In [ ]:

alt.Chart(ds).mark_rect().encode(
    alt.X('hours(start):O'),
    alt.Y('subAct'),
    alt.Color('durationNorm:Q', scale=alt.Scale(scheme='inferno')),
    row = 'DAY'
).properties(width=550).interactive()

In [ ]:
alt.Chart(ds).mark_boxplot().encode(
    x = 'durationNorm',
    y = 'subAct',
).properties(width = 550).interactive()

In [ ]:
alt.Chart(ds).mark_bar().encode(
    alt.X('durationNorm', bin = alt.Bin(maxbins=100)),
    y = 'count(subAct)',
    row = 'subAct'
).properties(width = 550).interactive()

In [ ]:
#https://altair-viz.github.io/gallery/stacked_bar_chart_sorted_segments.html
alt.Chart(ds).mark_bar().encode(
    x = 'count(subAct)',
    y = 'subAct',
    color = 'WDWE',
    column = 'WDWE' 
)

In [ ]:
#https://altair-viz.github.io/gallery/stacked_bar_chart_sorted_segments.html
alt.Chart(ds).mark_bar().encode(
    x = 'count(subAct)',
    y = 'subAct',
    color = 'DAY',
    row = 'WDWE' 
)

In [ ]:
#https://altair-viz.github.io/gallery/stacked_bar_chart_sorted_segments.html
alt.Chart(ds).mark_bar().encode(
    x = 'count(subAct)',
    y = 'subAct',
    color = 'DAY',
    column = 'WDWE' 
)

In [ ]:
#https://altair-viz.github.io/gallery/stacked_bar_chart_sorted_segments.html
# https://altair-viz.github.io/gallery/multiple_marks.html
alt.Chart(ds).mark_line(point=True).encode(
    y = 'count(subAct)',
    x = 'subAct',
    color = 'DAY',
    row = 'WDWE' 
)
# PLOT AS SOLID FILL

In [ ]:
# https://altair-viz.github.io/gallery/layered_area_chart.html
alt.Chart(ds).mark_area(opacity = 0.8).encode(
    y = 'count(subAct)',
    x = alt.X('subAct', axis = alt.Axis(labelAngle = -45)),
    color = 'DAY',
    row = 'WDWE' 
)
# PLOT AS SOLID FILL

In [ ]:
#https://altair-viz.github.io/gallery/stacked_bar_chart_sorted_segments.html
input_dropdown = alt.binding_select(options=['Mon','Tue','Wed','Thu','Fri','Sat','Sun'])
selection = alt.selection_single(fields=['DAY'], bind=input_dropdown, name='Test Name')
color = alt.condition(selection,
                    alt.Color('WDWE:Q', legend=None),
                    alt.value('lightgray'))

alt.Chart(ds).mark_bar().encode(
    x = 'count(subAct)',
    y = 'subAct',
    color = 'DAY',
    column = 'WDWE' 
).add_selection(
    selection
).transform_filter(
    selection
)


In [ ]:
# https://altair-viz.github.io/gallery/natural_disasters.html
alt.Chart(ds).mark_circle(
    opacity=0.8,
    stroke='black',
    strokeWidth=1
).encode(
    alt.X('date(start):O', axis=alt.Axis(labelAngle=0)),
    alt.Y('subAct:N', axis=alt.Axis(labelAngle=0)),
    alt.Size('count(subAct):Q',
        scale=alt.Scale(range=[0, 2000]),
        legend=alt.Legend(title='Annual Global Deaths')
    ),
    alt.Color('WDWE:N', legend=None)
).properties(
    width=600,
    height=800
).interactive()

In [ ]:
#date(start):O
    
    # https://altair-viz.github.io/gallery/natural_disasters.html
alt.Chart(ds).mark_circle(
    opacity=0.8,
    stroke='black',
    strokeWidth=1
).encode(
    alt.X('date(start):T', axis=alt.Axis(labelAngle=0)),
    alt.Y('subAct:N', axis=alt.Axis(labelAngle=0)),
    alt.Size('count(subAct):Q',
        scale=alt.Scale(range=[0, 2000]),
        legend=alt.Legend(title='Annual Global Deaths')
    ),
    alt.Color('WDWE:N', legend=None)
).properties(
    width=600,
    height=800
).interactive()

In [ ]:
#date(start):O
    
    # https://altair-viz.github.io/gallery/natural_disasters.html
alt.Chart(ds).mark_circle(
    opacity=0.8,
    stroke='black',
    strokeWidth=1
).encode(
    alt.X('start:T', axis=alt.Axis(labelAngle=0)),
    alt.Y('subAct:N', axis=alt.Axis(labelAngle=0)),
    alt.Size('count(subAct):Q',
        scale=alt.Scale(range=[0, 1000]),
        legend=alt.Legend(title='Annual Global Deaths')
    ),
    alt.Color('WDWE:N', legend=None)
).properties(
    width=600,
    height=800
).interactive()

In [ ]:
#https://altair-viz.github.io/gallery/stripplot.html
stripplot =  alt.Chart(ds, width = 40).mark_circle(size=8).encode(
    x=alt.X(
        'jitter:N',
        title=None,
        axis=alt.Axis(values=[0], ticks=True, grid=False, labels=False),
        scale=alt.Scale(),
    ),
    y=alt.Y('deltaStartEnd'),
    color=alt.Color('DAY:N', legend=None),
    column=alt.Column(
        'subAct:N',
        header=alt.Header(
            labelAngle=-90,
            titleOrient='top',
            labelOrient='bottom',
            labelAlign='right',
            labelPadding=0,
        ),
    ),
).transform_calculate(
    # Generate Gaussian jitter with a Box-Muller transform
    jitter='sqrt(-2*log(random()))*cos(2*PI*random())'
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
)

stripplot

In [ ]:
alt.Chart(
    ds,
    title="2010 Daily High Temperature (F) in Seattle, WA"
).mark_rect().encode(
    x='hoursminutes(start)',
    y='subAct:O',
    color=alt.Color('durationNorm', scale=alt.Scale(scheme="inferno")),
    #tooltip=[
     #   alt.Tooltip('monthdate(date):T', title='Date'),
      #  alt.Tooltip('max(temp):Q', title='Max Temp')
    #]
).properties(width=550)